This notebook is part of the *orix* documentation https://orix.rtfd.io. Links to the documentation won’t work from the notebook.

# Stereographic projection

In [ ]:
# Exchange "inline" for:
# "qt5" for interactive plotting from the pyqt package
# "notebook" for inline interactive plotting when running on Binder
%matplotlib inline

import matplotlib.pyplot as plt
from orix import vector, plot


# We'll want our plots to look a bit larger than the default size
new_params = {
    "figure.figsize": (10, 10),
    "lines.markersize": 10,
    "font.size": 15,
    "axes.grid": False,
}
plt.rcParams.update(new_params)

## Plot vectors

Default style

In [ ]:
v1 = vector.Vector3d([[0, 0, 1], [1, 0, 1], [1, 1, 1]])
fig, ax = plt.subplots(subplot_kw=dict(projection="stereographic"))
ax.scatter(v1)

Hover the cursor over the equatorial plane to see the spherical coordinates when
plotting interactively.

In [ ]:
plt.rcParams["axes.grid"] = True

### Upper and lower hemisphere

The axes' hemisphere *must* be set before calling `axes.scatter()`.

In [ ]:
v2 = vector.Vector3d([[1, 1, 2], [1, 1, -2]])

fig, ax = plt.subplots(ncols=2, subplot_kw=dict(projection="stereographic"))

ax[0].scatter(v2, c="C0")
ax[0].show_hemisphere_label()
ax[0].set_labels()

ax[1].hemisphere = "lower"  # /"south", or "upper"/"north"
ax[1].scatter(v2, c="C1")
ax[1].show_hemisphere_label()
ax[1].set_labels("RD", "TD", "ND")

### Control grid

Set polar and azimuth grid resolution, upon initialization or after the plot
was created

In [ ]:
subplot_kw = dict(
    projection="stereographic", polar_resolution=15, azimuth_resolution=30
)
fig, ax = plt.subplots(ncols=2, subplot_kw=subplot_kw)

ax[0].scatter(v1)

v3 = vector.Vector3d([[0, 0, -1], [-1, 0, -1], [-1, -1, -1]])
ax[1].hemisphere = "south"
ax[1].show_hemisphere_label()
ax[1].scatter(v3)
ax[1].polar_grid(30)
ax[1].azimuth_grid(45)

### Annotate vectors

In [ ]:
fig, ax = plt.subplots(ncols=2, subplot_kw=dict(projection="stereographic"))

ax[0].scatter(v1)
ax[0].show_hemisphere_label()
for vi in v1:
    ax[0].text(vi, s=vi._nice_string_repr())

v3 = vector.Vector3d([[0, 0, -1], [-1, 0, -1], [-1, -1, -1]])
ax[1].hemisphere = "lower"
ax[1].scatter(v3)
ax[1].show_hemisphere_label()
for vi in v3[:2]:
    ax[1].text(vi, s=vi._nice_string_repr("()"))
for vi in v3[2:]:
    ax[1].text(vi, s=vi._nice_string_repr("  "))

### Pass spherical coordinates

Instead of vectors

In [ ]:
fig, ax = plt.subplots(
    figsize=(5, 5), subplot_kw=dict(projection="stereographic")
)
azimuth = v1.phi
polar = v1.theta
ax.scatter(azimuth, polar)
ax.set_labels("RD", "TD", None)

print(azimuth, polar)

## Experimental functionality

Plot symmetry elements of point groups with proper rotations only

In [ ]:
marker_size = 500
fig, ax = plt.subplots(
    figsize=(10, 10), ncols=2, subplot_kw=dict(projection="stereographic")
)

ax[0].grid(False)
ax[0].set_title("432", pad=20)
# 4-fold (outer markers will be clipped a bit...)
v4fold = vector.Vector3d(
    [[0, 0, 1], [1, 0, 0], [-1, 0, 0], [0, 1, 0], [0, -1, 0]]
)
ax[0].symmetry_marker(v4fold, fold=4, c="C4", s=marker_size)
# 3-fold
v3fold = vector.Vector3d([[1, 1, 1], [1, -1, 1], [-1, -1, 1], [-1, 1, 1]])
ax[0].symmetry_marker(v3fold, fold=3, c="C3", s=marker_size)
# 2-fold
v2fold = vector.Vector3d([
    [1, 0, 1],
    [0, 1, 1],
    [-1, 0, 1],
    [0, -1, 1],
    [1, 1, 0],
    [-1, -1, 0],
    [-1, 1, 0],
    [1, -1, 0],
])
ax[0].symmetry_marker(v2fold, fold=2, c="C2", s=marker_size)

ax[1].grid(False)
ax[1].set_title("222", pad=20)
# 2-fold
v2fold = vector.Vector3d([
    [0, 0, 1], [1, 0, 0], [-1, 0, 0], [0, 1, 0], [0, -1, 0]
])
ax[1].symmetry_marker(v2fold, fold=2, c="C2", s=800)